In [1]:
import pandas as pd
import keras
from keras.layers import LSTM, SimpleRNN, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
from keras.utils.np_utils import to_categorical
from keras.regularizers import l2
import tensorflow as tf
import numpy as np
import os
import fnmatch
from matplotlib import pyplot as plt
from IPython.display import clear_output
from modules.plot_losses import PlotLosses
from modules.lstm_encoder import LSTMEncoder

Using TensorFlow backend.


## Training

In [4]:
layers = [
    Dropout(0.2),
    Dense(4096, activation='relu'),
    Dropout(0.2),
    Dense(2048, activation='relu'),
    Dense(2048, activation='relu'),
    Bidirectional(LSTM(1024, return_sequences=True)),
]

In [6]:
encoder = LSTMEncoder(
    hidden_layers=layers,
    source_path='./features/lstm/2_steps/',
    lr=0.00001,
)

In [7]:
encoder.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 6144)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 6144)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 2, 4096)           25169920  
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 4096)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 2, 2048)           8390656   
_________________________________________________________________
dense_3 (Dense)              (None, 2, 2048)           4196352   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2, 2048)           25174016  
__________

In [8]:
plot_losses = PlotLosses()
checkpoint = keras.callbacks.ModelCheckpoint(
    './convnet_weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='auto', 
    period=1,
)
callbacks = [plot_losses, checkpoint]

In [9]:
encoder.train(100, callbacks)

Epoch 1/100
 47/101 [============>.................] - ETA: 1:27 - loss: 4.8133 - categorical_accuracy: 0.0533

KeyboardInterrupt: 

## Encoding

In [ ]:
encoder = LSTMEncoder(
    source_path='./features/lstm/2_steps/',
    regularizer=l2, 
    lr=0.000001,
    weights_path='./convnet_weights/lstm/base/weights.19-1.13.hdf5',
)

In [ ]:
_ = encoder.encode('train', './features/lstm/encoded/features_train.npy')
_ = encoder.encode('dev', './features/lstm/encoded/features_dev.npy')

## Predictions

In [ ]:
encoder = LSTMEncoder(
    source_path='./features/lstm/2_steps/',
    regularizer=l2, 
    lr=0.000001,
    weights_path='./convnet_weights/lstm/base/weights.19-1.13.hdf5',
)

In [ ]:
encoder.predict('dev')